In [ ]:
import pandas as pd
import regex as re
from constants import * #file with locations to replace
from tqdm import tqdm
import geograpy4
import numpy as np

## Replace or precise locations

In [ ]:
locations = pd.read_csv("path") #csv file with all locations from tweets and corresponding tweets ids
locations = locations.dropna(subset=["place"])
locations = locations.reset_index().drop(["index"],axis=1)

#Replace tweets fake locations by empty string and contractions by full location
all_keys = list(replace_dict.keys())
all_keys.sort(key=len)
all_keys = all_keys[::-1]
all_keys_lower = [x.lower() for x in all_keys]

all_results = [replace_dict[x] for x in all_keys]

new_locations = []
for location in locations['place']:
    new_loc_str = re.sub("{", "" , location)
    new_loc_str = re.sub("}", "" , new_loc_str)
    loc_lower = new_loc_str.lower()
    for i, key in enumerate(all_keys_lower):
        try:
            p=re.compile(r"\b"+key+r"\b")
            
            m = p.search(loc_lower)
            while m is not None :
                span = m.span()
                new_loc_str = new_loc_str[:span[0]]+ \
                        "{"+str(i)+"}" + \
                        new_loc_str[span[1]:]
                loc_lower = new_loc_str.lower()
                m = p.search(loc_lower)
            
            
        except AttributeError:
            continue
    
    new_loc_str = new_loc_str.format(*all_results)
    new_locations.append(new_loc_str)

new_locations_df = pd.DataFrame(new_locations)
result = pd.concat([locations, new_locations_df], axis=1)
result = result.drop(["Unnamed: 0"], axis=1)
result.to_csv("path") #export old locations with the new "cleaned" ones

## Precise locations with geograpy

In [ ]:
tweets=pd.read_parquet(r"path_to_dataset") #dataset with full tweets
tweets=tweets.reset_index().drop(["Unnamed: 0","index"],axis=1)
#Create fields to store locations
tweets["location"]=""
tweets["latitude"]=""
tweets["longitude"]=""

list_location=tweets["user_location"].value_counts()
list_location=list_location[list_location>1]
list_location=list(list_location.index)

In [ ]:
places_to_geolocate=pd.read_csv(r"path_csv_with_new_locations")
places_to_geolocate=places_to_geolocate.drop(["Unnamed: 0"], axis=1)
places_to_geolocate["geograpy_output"]=""

#Use geograpy to get place context of each user location we precised
for i in tqdm(range(0,len(places_to_geolocate.index))):
    try:
        places_to_geolocate["geograpy_output"][i] = geograpy4.get_place_context(places_to_geolocate["0"][i])
    except:
        pass

places_to_geolocate.geograpy_output = places_to_geolocate.geograpy_output.apply(lambda y: np.nan if len(y)==0 else y)
places_to_geolocate=places_to_geolocate.dropna(subset=["geograpy_output"]).reset_index().drop(["index"],axis=1)

In [ ]:
#Allocate location longitude and latitude to corresponding location if full dataset
for i in tqdm(range(0,len(places_to_geolocate.index))):
    list_to_geolocate=tweets.index[tweets['user_location'] == places_to_geolocate["place"][i]].tolist()
    for j in list_to_geolocate:
        tweets["location"][j]=places_to_geolocate["geograpy_output"][i][0]["display_name"]
        tweets["latitude"][j]=places_to_geolocate["geograpy_output"][i][0]["lat"]
        tweets["longitude"][j]=places_to_geolocate["geograpy_output"][i][0]["lon"]


    
tweets.location = tweets.location.apply(lambda y: np.nan if len(y)==0 else y)
tweets=tweets.dropna(subset=["location"]).reset_index().drop(["index"],axis=1)

In [ ]:
#Export new dataset with location, latitude and longitude
tweets.to_csv("")